In [33]:
import os
import torch
import pickle
from transformers import BertTokenizer, BertModel, CamembertModel, CamembertTokenizer, AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Avec BERT (pas au point)

In [36]:
def get_document_embeddings_batch(texts, model, tokenizer, batch_size=8):
    vectors = []
    for text in texts:
        input_ids = tokenizer(text, return_tensors="pt", truncation = True, padding = True)
        with torch.no_grad():
            outputs = model(**input_ids)
        vectors.append(outputs.last_hidden_state.mean(dim = 1))

    return torch.cat(vectors, dim=0)

def semantic_search(query, model, tokenizer, document_embeddings, n_results=5):
    query_embedding = get_document_embeddings_batch(query, model, tokenizer, batch_size=8)
    similarity_scores = cosine_similarity(query_embedding, document_embeddings)[0]
    return(similarity_scores)
    # sorted_indices = similarity_scores.argsort()
    # return(sorted_indices[::-1][:n_results])

def save_embeddings(embeddings, output_file):
    with open(output_file, 'wb') as f:
        pickle.dump(embeddings, f)

def load_embeddings(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

In [32]:
tokenizer.model_max_length

1000000000000000019884624838656

In [45]:

if __name__ == "__main__":
    # Replace 'bert-base-uncased' with any other suitable pre-trained model if needed.
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # model = BertModel.from_pretrained('bert-base-uncased')
    modelname = 'camembert/camembert-base'
    model = AutoModel.from_pretrained("camembert-base")
    tokenizer = AutoTokenizer.from_pretrained("camembert-base")
    
    embeddings_file = "document_embeddings.pkl"

    data_folder = "./data/Texte/"
    n_results = 5  # Change this value to get a different number of relevant documents.

    # Load and process your text data from separate folders.
    documents = []
    for folder_name in os.listdir(data_folder):
        folder_path = os.path.join(data_folder, folder_name)
        if not os.path.isdir(folder_path):
            continue
        for file_name in os.listdir(folder_path):
            if file_name == "full.txt" :
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()
                    documents.append(text)
    
    # Get embeddings for all the documents.
    if os.path.exists(embeddings_file):
        document_embeddings = load_embeddings(embeddings_file)
    else:
        batch_size = 100
        document_embeddings = get_document_embeddings_batch(documents, model, tokenizer, batch_size)
        save_embeddings(document_embeddings, embeddings_file)
    

    

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([10, 768])

In [44]:
# Example query.
query = "Hopital public"

# Get the most relevant documents.
relevant_indices  = semantic_search(query, model, tokenizer, document_embeddings, n_results)
# Print the most relevant documents.
res = pd.DataFrame(zip(documents, relevant_indices), columns = ["Doc", "Cos"])
res
# print("Most relevant documents:")
# for i, index in enumerate(relevant_indices):
#     print(documents[index])
#     print("------------------------")

,Doc,Cos
0,Action publique : comment penser la solidarité...,0.196980
1,Vers une reconquête de la qualité de l’air : p...,0.181160
2,Changement de comportements\n« Évitez de mange...,0.251972
3,La reconnaissance de la Halle Tony Garnier\nRé...,0.246271
4,Accompagner les start-up\nAxeleo accompagne de...,0.090111
5,Le mentorat et la création d'entreprise : l'ex...,0.128467
6,L'esprit start-up : l'exemple de Dougs\nCréée ...,0.143243
7,Allier PME et start-up\nLa CPME (Confédération...,0.200795
8,"Veille M3 / Cédric Van Styvendael, VP à la cul...",0.217256
9,Mesurer le bien-être pour mieux aménager les v...,0.088114


In [78]:
document_embeddings

tensor([[-0.5240,  0.1943, -0.0209,  ..., -0.2163,  0.1950,  0.3014],
        [-0.4689,  0.1753,  0.0157,  ..., -0.1821,  0.1726,  0.2304],
        [-0.3280,  0.2175,  0.0427,  ..., -0.2058,  0.0920,  0.1564],
        ...,
        [-0.3780,  0.2710,  0.0988,  ..., -0.0860,  0.1675,  0.2656],
        [-0.3799,  0.3006,  0.1161,  ..., -0.1333,  0.2829,  0.1215],
        [-0.4635,  0.2810,  0.0787,  ..., -0.1115,  0.1915,  0.2522]])

# AVEC TF-IDF

In [156]:
from sklearn.feature_extraction.text import CountVectorizer

data_folder = "./data/Texte/"
documents=[]
nom_docs=[]
for folder_name in os.listdir(data_folder):
    # print(folder_name)
    folder_path = os.path.join(data_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue
    for file_name in os.listdir(folder_path):
        # print(file_name)
        if file_name == "texte.txt":
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
                documents.append(text)
                nom_docs.append(folder_path.split('/')[3])
                # print(folder_path.split('/')[3], text)
vectorizer=CountVectorizer(lowercase=False)
nOcc=vectorizer.fit_transform(documents)
nOcc=pd.DataFrame.sparse.from_spmatrix(nOcc,index=nom_docs,columns=vectorizer.get_feature_names_out())

In [172]:
query = vectorizer.transform(["musique art culture"]) #Process (lemmatize + stopwords)

similarities = cosine_similarity(query, nOcc)

# Classement des documents en fonction de leur similarité avec la requête
sorted_indices = similarities
sim = pd.DataFrame(sorted_indices[0], nom_docs).sort_values(by=0, ascending=False)

In [185]:
txt = []
for doc in sim[:5].index:
    with open(f"./data/Texte/{doc}/full.txt", "r", encoding="utf-8") as f:
        texte = f.read()
        print(texte)
        txt.append(texte)
    print("----------------------")

Lyon et la question des « arts »
nan
Interroger les singularités de l'identité culturelle lyonnaise ne peut se faire sans que l'on revisite les différentes acceptions qu'a pu revêtir le terme " art ". Le terme a été référé initialement au champ de l'utilité? bien avant de désigner celui de la spéculation intellectuelle ou celui du beau et du désintéressement. La question se pose de savoir si, à l'orée du 21e siècle, Lyon peut mettre ses institutions au service d'une actualisation des trois sens du terme.Sommaire :1 Pourquoi Lyon n’a pu servir la cause des «  arts libéraux »Mais tout d’abord qu’appelle-t-on « arts libéraux » ?Pourquoi Lyon ne s’est-elle pas illustrée dans les « arts libéraux » ?2 Pourquoi Lyon a remarquablement servi la cause des « arts mécaniques », mais sur un mode éminemment paradoxal Les « arts  mécaniques, arts de transformation de la matièreMais un double paradoxe ne peut pas ne pas être relevéLe contre-exemple nancéen3 Pourquoi Lyon a servi la cause des « beaux-a

### Resumé avec ChatGPT

In [194]:
import openai

with open("APIKEY", "r", encoding="utf-8") as f:
    openai.api_key = f.read()

openai.organization = "org-mqd2akvmI5TNokavyM3mWXSq"
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages = [{"role": "user", 
                 "content": f"""Résume l'ensemble de ces différents textes séparés par ce signe "---" en 2 phrases : {"---".join(txt)}"""}]
    )
print(chat_completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Lyon a connu une \u00e9volution dans sa vision des arts, passant d'une utilisation limit\u00e9e dans le champ de l'utilit\u00e9 \u00e0 une reconnaissance de l'importance des arts m\u00e9caniques, des beaux-arts et de leur r\u00f4le dans l'enseignement. Cependant, il reste encore des limites \u00e0 l'expansion de l'art dans l'enseignement, notamment dans l'enseignement sup\u00e9rieur o\u00f9 les disciplines restent cloisonn\u00e9es, et les politiques d'art public sont encore timides dans l'agglom\u00e9ration lyonnaise malgr\u00e9 quelques initiatives.",
        "role": "assistant"
      }
    }
  ],
  "created": 1689850633,
  "id": "chatcmpl-7eLdJggwqVaiWoAkYv8re1zmNigA7",
  "model": "gpt-3.5-turbo-16k-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 132,
    "prompt_tokens": 9168,
    "total_tokens": 9300
  }
}


In [195]:
print(chat_completion["choices"][0]["message"]["content"])

Lyon a connu une évolution dans sa vision des arts, passant d'une utilisation limitée dans le champ de l'utilité à une reconnaissance de l'importance des arts mécaniques, des beaux-arts et de leur rôle dans l'enseignement. Cependant, il reste encore des limites à l'expansion de l'art dans l'enseignement, notamment dans l'enseignement supérieur où les disciplines restent cloisonnées, et les politiques d'art public sont encore timides dans l'agglomération lyonnaise malgré quelques initiatives.
